In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

import os
import glob
import time
import datetime as dt
from netCDF4 import Dataset as nc_Dataset
from netCDF4 import date2num, num2date
import pandas as pd
import numpy as np
import xarray as xr
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.markers import MarkerStyle
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from HRRR_URMA_Datasets import *
from SR_UNet_simple import *

In [ ]:
urma_sp_rest = xr.open_dataset("/data1/projects/RTMA/alex.schein/URMA_train_test/LOOSE_FILES/train/t2m/urma_20210101_t00z.nc", decode_timedelta=True)
urma_orig = xr.open_dataset("/scratch/RTMA/alex.schein/urma_aidatadepot_test.grb2_wexp", engine="cfgrib", decode_timedelta=True)
urma_resetradius = xr.open_dataset("/scratch/RTMA/alex.schein/urma_resetradius.grib2", engine="cfgrib", decode_timedelta=True)

hrrr_orig = xr.open_dataset(f"/scratch/RTMA/alex.schein/test_hrrr_newspacing_6370000.grib2", engine='cfgrib', decode_timedelta=True)
hrrr_6371950 = xr.open_dataset(f"/scratch/RTMA/alex.schein/test_u10m.grib2", engine='cfgrib', decode_timedelta=True)

In [ ]:
IDX_MIN_LON = 595+1 #NOTE: different from URMA grid! 
IDX_MIN_LAT = 645

IMG_SIZE_LON = 180
IMG_SIZE_LAT = 180

In [ ]:
# (7/4) Note that the new HRRR indexing matches URMA indexing because it's now on the wexp grid, i.e. +200 lon indices
hrrr_sp_rest = hrrr_orig.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                        #x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))
                              x=slice(796, 796+IMG_SIZE_LON))

hrrr_6371950_sp_rest = hrrr_6371950.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                        x=slice(IDX_MIN_LON, IDX_MIN_LON+IMG_SIZE_LON))

urma_orig_sp_rest = urma_orig.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                   x=slice(796, 796+IMG_SIZE_LON))

urma_resetradius_sp_rest = urma_resetradius.isel(y=slice(IDX_MIN_LAT, IDX_MIN_LAT+IMG_SIZE_LAT),
                                                 x=slice(796, 796+IMG_SIZE_LON))


projection = ccrs.LambertConformal(central_longitude = 265-360,
                                    central_latitude = 38.5,
                                    standard_parallels = (38.5,38.5),
                                    cutoff = 0)

fig = plt.figure(figsize=(9,9))
ax1 = fig.add_subplot(1,1,1, projection=projection)

ax1.set_extent([252-360, 252.2-360, 39, 39.2])
ax1.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1)
ax1.add_feature(cfeature.STATES.with_scale('50m'))
mks = MarkerStyle('o', fillstyle="none")

# ax1.scatter(urma_orig_sp_rest.longitude.data, urma_orig_sp_rest.latitude.data, transform=ccrs.PlateCarree(), s=50, c='r', marker=mks, label='URMA original')
ax1.scatter(urma_resetradius_sp_rest.longitude.data, urma_resetradius_sp_rest.latitude.data, transform=ccrs.PlateCarree(), s=50, c='g', marker=mks, label='URMA reset radius')
ax1.scatter(hrrr_sp_rest.longitude.data, hrrr_sp_rest.latitude.data, transform=ccrs.PlateCarree(), s=50, c='k', marker =mks, label='HRRR 6370000 new spacing')
# ax1.scatter(hrrr_6371950_sp_rest.longitude.data, hrrr_6371950_sp_rest.latitude.data, transform=ccrs.PlateCarree(), s=50, c='b', marker =mks, label='HRRR R=6371950')


plt.title("Grid alignment, URMA vs Regridded HRRR")
plt.legend(loc='lower left')
# plt.savefig("grid_alignment.png", dpi=200, bbox_inches='tight')


In [ ]:
difference_longitude = np.abs((hrrr_sp_rest.longitude - urma_resetradius_sp_rest.longitude).data)
difference_latitude = np.abs((hrrr_sp_rest.latitude - urma_resetradius_sp_rest.latitude).data)
difference_sqrt = np.sqrt(np.square(difference_latitude)+np.square(difference_longitude))
np.max(difference_sqrt)

In [ ]:
difference_longitude = np.abs((hrrr_6371950_sp_rest.longitude - urma_sp_rest.longitude).data)
difference_latitude = np.abs((hrrr_6371950_sp_rest.latitude - urma_sp_rest.latitude).data)
difference_sqrt = np.sqrt(np.square(difference_latitude)+np.square(difference_longitude))
np.max(difference_sqrt)

In [ ]:
urma_resetradius_sp_rest.longitude.data

In [ ]:
hrrr_sp_rest.longitude.data